In [1]:
# https://automl.github.io/auto-sklearn/stable/
import numpy as np
import pandas as pd
import os


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_x.csv') 
sample_submission = pd.read_csv('sample_submission.csv') 

In [3]:
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [4]:
X_train = train['text'] 
y_train = train['author'] 
X_test = test['text']

In [5]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test = np.asarray(X_test)

In [6]:
import autosklearn.classification

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [7]:
automl = autosklearn.classification.AutoSklearnClassifier()


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape, X_train.shape

((54879, 34719), (54879,))

In [9]:
from sklearn.feature_extraction.text import  TfidfTransformer
tf_transformer = TfidfTransformer()
X_train_tfidf = tf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(54879, 34719)

In [10]:
automl.fit(X_train_tfidf, y_train)

[ERROR] [2020-11-22 23:02:59,035:AutoMLSMBO(1)::c95dc36d77bf77e4965b42cd3f2f6fe2] Error getting metafeatures: Can't pickle local object 'Client.__init__.<locals>.<lambda>'
[ERROR] [2020-11-22 23:02:59,057:AutoMLSMBO(1)::c95dc36d77bf77e4965b42cd3f2f6fe2] Error getting metafeatures (encoded) : Can't pickle local object 'Client.__init__.<locals>.<lambda>'


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


AutoSklearnClassifier(dask_client=None,
                      delete_output_folder_after_terminate=True,
                      delete_tmp_folder_after_terminate=True,
                      disable_evaluator_output=False, ensemble_nbest=50,
                      ensemble_size=50, exclude_estimators=None,
                      exclude_preprocessors=None, get_smac_object_callback=None,
                      include_estimators=None, include_preprocessors=None,
                      initial_configurations_via_metalearning=25,
                      load_models=None, logging_config=None,
                      max_models_on_disc=50, memory_limit=3072,
                      metadata_directory=None, metric=None, n_jobs=None,
                      output_folder=None, per_run_time_limit=360,
                      resampling_strategy='holdout',
                      resampling_strategy_arguments=None, seed=1,
                      smac_scenario_args=None, time_left_for_this_task=3600,
             

In [11]:
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tf_transformer.transform(X_test_counts)


In [12]:
# y_pred = automl.predict(X_test_tfidf)
y_pred_prob = automl.predict_proba(X_test_tfidf)


In [13]:
y_pred_prob.shape 

(19617, 5)

In [14]:
sample_submission[['0','1','2','3','4']] = y_pred_prob 
sample_submission.head()

,index,0,1,2,3,4
0,0,0.193903,0.218500,0.201466,0.195358,0.190774
1,1,0.198865,0.208478,0.197455,0.197876,0.197325
2,2,0.227889,0.194316,0.194694,0.191368,0.191733
3,3,0.201580,0.187433,0.221007,0.190284,0.199696
4,4,0.202430,0.203660,0.199046,0.200153,0.194711


In [16]:
sample_submission.to_csv('automl_test.csv',index=False)